# Intelligent Systems

## Academic year 2024-2025

### Lab 1: State space search
Carmen María Noblejas Carreto
Lucian Andrei Negoita

## 1. INTRODUCTION:

We were asked to design an algorithm that is capable of optimizing the transportation of a person from a place of origin to a specific destination within a city. In this scenario, the autonomous vehicle will have to navigate a network of urban streets and intersections, where all routes are potentially valid.

The system must optimize the path selection not only to find a valid route, but also to minimize the travel time.

## 2. PROBLEM DESCRIPTION:

We have to solve a problem in which an autonomout vehicle must find the fastest route between any two intersections in a city. The search space is defined by an urban road system where the vehicle can move in several directions to reach its destination.

The problem can be defined as: 

 * **Initial state**: A starting point that represents the vehicle's initial intersection.
 * **States**: All intersections in the city are valid for traffic and can be visited by the vehicle.
 * **Final state**: Arriving at the destination intersection.
 * **Actions**: Moving from one intersection to another (that must be linked) through the city streets.



## 3. DEVELOPMENT OF THE LAB ASSIGMENT

We have been provided with a set of problem intances with different dimensionalities, and the algorithm must be efficient enough to work correctly with all the instances provided.


### 3.1 Input Problems

For each scenario we are given a json file that contains the following information:

 * **ADDRESS**: Address used.
 * **DISTANCE**: Maximum radius to draw intersections and segments around the address.
 * **INTERSECTIONS**: List of dictionaries with information on intersections.
 * **SEGMENTS**: List of dictionaries with information about segments, i.e., streets between two intersections.
 * **INITIAL**: Initial intersection.
 * **FINAL**: Final intersection.

 In each dictionary in **INTERSECTIONS**, there are three keys:
 
 * **IDENTIFIER**
 * **LONGITUDE**
 * **LATITUDE**
 
 In each dictionary in **SEGMENTS**, there are four keys:
 
 * **ORIGIN**
 * **DESTINATION**
 * **DISTANCE**
 * **SPEED**

## 4. WORK PLAN

## 4.1 Tasks

* State space design:
    * Description how the state space, actions, and cost of actions are represented


### CLASS STATE

In [16]:
class State:
    def __init__(self, identifier, latitude, longitude):
        self.identifier = identifier
        self.latitude = latitude
        self.longitude = longitude
        self.neighbors = []  # List of (neighbor_state, action/segment) tuples

    def __eq__(self, other):
        return isinstance(other, State) and self.identifier == other.identifier

    def __hash__(self):
        # Hash method to allow using State instances in sets or as dictionary keys
        return hash(self.identifier)

    def __repr__(self):
        return f"State({self.identifier}, {self.latitude}, {self.longitude})"


##### Purpose:
* This class represents an intersection (state) in the city graph, it stores its unique identifier, geographical coordinates, and connected neighbors

##### Attributes:
* `Identifier`
* `Latitude, longitude`: Geographical coordinates of the intersection
* `Neighbours`: List of tuples with "neighboring" `States` objects and the `Actions` ofjects (road segments)

##### Methods:
* `__eq__(self, other)`: Determines the equality based on the state's identifier.
* `__hash__(self)` : Emables `State`objects to be used in sets and as dictionary keys. Returns a hash of the identifier.
* `__repr__(self)` : Returns a string representation of the `State` object (debugging purpose)

It is important to take into account that we have pre-sorted the Neighbors during initialization in the class `ImportJSON.py` for consistent traversal during searches. (We will see this class later on the notebook).

### CLASS ACTION

In [17]:
class Action:
    def __init__(self, origin, destination, distance, speed):
        self.origin = origin  # State object
        self.destination = destination  # State object
        self.distance = distance  # Distance between origin and destination
        self.speed = speed / 3.6  # Speed limit on this segment

    def cost(self):
        # Calculate travel time as the cost
        return self.distance / self.speed

    def __repr__(self):
        return f"Action({self.origin.identifier} -> {self.destination.identifier}, distance={self.distance}, speed={self.speed})"


##### Purpose:
* This class represents a road segment that connects two intersections (`States`) in the city. It includes information about the distance and speed limit.

##### Attributes:
* `Origin`: Starting state
* `Destination`: End state of the action 
* `Distance`: Between `origin` and `destination` (meters)
* `Speed`: Speed limit of the segment (converted to meters per second)

##### Methods:
* `cost(self)`: Returns the travel time and the calculated cost by dividing `distance` by the `speed` (seconds).
* `__repr__(self)`: Provides a string representation of the `Action` object for easy debugging and logging.

##### IMPORTANT THINGS TO TAKE INTO ACCOUNT:
* Speed is converted from kilometers per hour (km/h) to meters per second (m/s) by dividing by `3.6`. Ensuring consistent units when calculating the cost.
* The cost is calculated in seconds, reflecting the travel time for the segment.

### CLASS NODE

In [18]:
class Node:

    node_counter = 0  # Couter to know the iteration of creation for sorting

    def __init__(self, state, parent=None, action=None, path_cost=0, heuristic_cost=0):
        self.state = state  # State object
        self.parent = parent  # Parent Node
        self.action = action  # Action taken to reach this node
        self.path_cost = (
            path_cost  # Total cost from the root node to this node (g_cost)
        )
        self.heuristic_cost = heuristic_cost  # Heuristic estimate to the goal (h_cost)
        self.f_cost = (
            self.path_cost + self.heuristic_cost
        )  # Total cost for A* (f = g + h)
        self.depth = 0 if parent is None else parent.depth + 1
        self.order = Node.node_counter  # For sorting
        Node.node_counter += 1

    def __lt__(self, other):
        return (self.f_cost, self.state.order) < (other.f_cost, other.order)

    def __repr__(self):
        return f"Node(State={self.state.identifier}, Path cost={self.path_cost}, Heuristic={self.heuristic_cost}, Depth={self.depth})"


##### Purpose:
* Represents a node in the search tree. It contains information about the current state, the action taken to reach it, and the cumulative cost and depth of the node.

##### Attributes:
* `State`: Current state (intersection)
* `Parent`: Parent node leadeing to the current one
* `Action`: Action taken to reach this node from the parent one
* `Path_cost`: Cummulative cost from the root to this node (seconds)
* `Heuristic_cost`: Heuristic estimated to the goal
* `F_cost`: Total cost for A*: (f = g + h)
* `Depth`: Depth of the node in the search tree
* `Order`: We store the iteration in which the node is created to sort it. We asign to it the counter.
* `Counter`: Counter to know the iteration in which the node is created.

##### Methods:
* `__repr__(self)`: Provides a string representation of the `Node`object, displaying the satate identifier, path cost, and depth for debugging 
* `__lt__(self, other)`: Comparison based on `f_cost` and then `id` for tie-breaking

##### NOTE:
* The `path_cost` represents the total travel time in seconds, accumulating the costs from the `Action` objects.

### CLASS PROBLEM

In [19]:
class Problem:
    def __init__(self, initial_state, goal_state, intersections, segments):
        self.initial_state = initial_state  # Starting State
        self.goal_state = goal_state  # Goal State
        self.intersections = (
            intersections  # Dictionary of State objects (key: identifier)
        )
        self.segments = segments  # List of Action objects

    def actions(self, state):
        """Returns the actions available from a given state."""
        return state.neighbors

    def result(self, state, action):
        """Returns the resulting state after taking an action."""
        return action.destination

    def goal_test(self, state):
        """Checks if the given state is the goal state."""
        return state == self.goal_state

    def path_cost(self, cost_so_far, state1, action, state2):
        """Returns the cost of a solution path that arrives at state2 from state1."""
        return cost_so_far + action.cost()


##### Purpose:
* Represents the problem of navigating the city by defining the initial state, goal state, and methods to interact with states and actions

##### Attributes:
* `Initial_state`: Statirng state for the search 
* `Goal_state`: Target state that the search aims to reach
* `Intersections`: Dictionary of all `State`objects, keyed by their identifier
* `Segments`: List of all `Action` objects (road segments).

##### Methods: 
* `actions(self, state)`: Returns available actions (segments) for the given state
* `result(self, state, action)`: Returns the state resulting from taking an action
* `goal_test`: Checks if a state is the goal state
* `path_cost`: Computes the cumulative cost of a path

### CLASS ImportJSON

In [20]:
import json
#from State import State
#from Action import Action
#from Problem import Problem


def loadJSON(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)

    intersections = {}
    for i_data in data["intersections"]:
        inter = State(
            identifier=i_data["identifier"],
            latitude=i_data["latitude"],
            longitude=i_data["longitude"],
        )
        intersections[inter.identifier] = inter

    segments = []
    for seg_data in data["segments"]:
        origin = intersections[seg_data["origin"]]
        destination = intersections[seg_data["destination"]]
        segment = Action(
            origin=origin,
            destination=destination,
            distance=seg_data["distance"],
            speed=seg_data["speed"],
        )
        segments.append(segment)
        # Add neighbors to the origin state
        origin.neighbors.append((destination, segment))

    # Pre-sort neighbors for each state
    for state in intersections.values():
        state.neighbors.sort(key=lambda x: x[0].identifier)

    initial_state = intersections[data["initial"]]
    goal_state = intersections[data["final"]]

    return Problem(initial_state, goal_state, intersections, segments)


##### Purpose:
* Loads and parses the JSON file that contains information about intersections and road segments. It creates the objects `State` and `Action` based on the data of the file and returns a `Problem` instance.

##### Methods
* `loadJSON(file_path)`: This method reads the JSON file, initializes `State` and `Action` objects, links intersections with their neighborsm pre-sorts the neighbors for each state (as we have said in the `State` class), and returns a `Problem` instance that contains all the required information for the search algorithm.

#### In this class we see some key operations...
* `Parsing Intersections`: Creates objects `State` for each intersection and stores them in a dictionary (easy access).
* `Parsing Segment`: Creates objects `Action` for each road segment, links origin and destination `State`objects.
* `Linking Neighbors`: Appends each destination and corresponging action to the `neighbors` list of the origin state.
* `Pre-sort Neighbors`: Sorts the `neighbors`list according on the destination identifier taking into account the iteration in which the node was generated.



### CLASS SEARCH 

Here we implement the different search algorithms: two uninformed and two informed (using the appropiate heuristics to find optimal routes) 

In [21]:
import time
import heapq
from geopy.distance import geodesic
from datetime import timedelta
from collections import deque
from abc import ABC, abstractmethod
#from Node import Node


First of all we import all the needed libraries to carry out the different search algorithms

It is important to see that as we are working with a notebook be don't need to import the different classes due to the fact that we are working in the same file, but in out .py we can see that we need to import the different classes from and to the different other classes where we want to use them.

In [22]:


def solution(node):
    path = []
    while node.parent is not None:
        path.append(node.action)
        node = node.parent
    path.reverse()
    return path



def format_solution_details(solution_path, stats):
    formatted_solution = []
    for action in solution_path:
        formatted_solution.append(
            f"{action.origin.identifier} → {action.destination.identifier}, {action.cost():.5f}"
        )
    formatted_solution_str = "[" + ", ".join(formatted_solution) + "]"

    formatted_output = (
        f"Generated nodes: {stats['nodes_generated']}\n"
        f"Expanded nodes: {stats['nodes_explored']}\n"
        f"Execution time: {stats['execution_time']}\n"
        f"Solution length: {stats['solution_depth']}\n"
        f"Solution cost: {stats['solution_cost']}\n"
        f"Solution: {formatted_solution_str}"
    )
    return formatted_output


def format_time(seconds):
    td = timedelta(seconds=seconds)
    days = td.days
    hours, remainder = divmod(td.seconds, 3600)
    minutes, sec = divmod(remainder, 60)
    microseconds = td.microseconds

    if days > 0:
        return f"{days}:{hours}:{minutes:02}:{sec:02}.{microseconds:06}"
    else:
        return f"{hours}:{minutes:02}:{sec:02}.{microseconds:06}"


In this last code cell we have different methods that will help us in the readability and to understand the output of the different algorithms:
* `Solution`: This is a helper solution that extracts the solution path.
* `Format_solution_details`: This method formattes the output for easier readability, showing us in a sorted way the generated nodes, expanded nodes, execution time, solution length, solution cost and the path solution.
* `Format_time`: This method will convert time to the desired format -> days:hours:seconds

#### BREADTH-FIRST SEARCH (BFS) FUNCTION

In [23]:


# Breadth-First Search (BFS) function
def breadth_first_search(problem):
    start_time = time.perf_counter()
    frontier = deque([Node(problem.initial_state)])
    explored = set()
    nodes_generated = 1
    nodes_explored = 0

    while frontier:
        node = frontier.popleft()
        nodes_explored += 1

        if problem.goal_test(node.state):
            solution_path = solution(node)
            execution_time = format_time(time.perf_counter() - start_time)
            solution_cost = format_time(node.path_cost)
            return solution_path, {
                "nodes_generated": nodes_generated,
                "nodes_explored": nodes_explored,
                "solution_depth": node.depth,
                "solution_cost": solution_cost,
                "execution_time": execution_time,
            }

        explored.add(node.state)

        for next_state, action in node.state.neighbors:
            if next_state not in explored and all(
                front_node.state != next_state for front_node in frontier
            ):
                child = Node(next_state, node, action, node.path_cost + action.cost())
                frontier.append(child)
                nodes_generated += 1

    return None, {
        "nodes_generated": nodes_generated,
        "nodes_explored": nodes_explored,
        "solution_depth": None,
        "solution_cost": None,
        "execution_time": format_time(time.perf_counter() - start_time),
    }


Implements the BFS algorithm using a queue (`deque`). It tracks nodes generated and explored, and returns the solution path and statistics upon finding the goal.

#### DEPTH-FIRST SEARCH (DFS) FUNCTION

In [24]:


# Depth-First Search (DFS) function
def depth_first_search(problem):
    start_time = time.perf_counter()
    frontier = [Node(problem.initial_state)]
    explored = set()
    nodes_generated = 1
    nodes_explored = 0

    while frontier:
        node = frontier.pop()
        nodes_explored += 1

        if problem.goal_test(node.state):
            solution_path = solution(node)
            execution_time = format_time(time.perf_counter() - start_time)
            solution_cost = format_time(node.path_cost)
            return solution_path, {
                "nodes_generated": nodes_generated,
                "nodes_explored": nodes_explored,
                "solution_depth": node.depth,
                "solution_cost": solution_cost,
                "execution_time": execution_time,
            }

        explored.add(node.state)

        for next_state, action in node.state.neighbors:
            if next_state not in explored:
                child = Node(next_state, node, action, node.path_cost + action.cost())
                frontier.append(child)
                nodes_generated += 1

    execution_time = format_time(time.perf_counter() - start_time)

    return None, {
        "nodes_generated": nodes_generated,
        "nodes_explored": nodes_explored,
        "solution_depth": None,
        "solution_cost": None,
        "execution_time": execution_time,
    }



Implements the DFS algorithm using a stack (list). It uses a `frontier_states`set for efficient duplicate checks and returns the solution path and statistics upon finding the goal.

#### A* SEARCH FUNCTION

In [25]:

# A* Search function
def a_star_search(problem, heuristic):
    start_time = time.perf_counter()
    frontier = []
    start_node = Node(
        problem.initial_state,
        heuristic_cost=heuristic(problem.initial_state, problem.goal_state),
    )
    heapq.heappush(frontier, (start_node.f_cost, start_node.order, start_node))

    explored = set()
    frontier_state_costs = {
        problem.initial_state: 0
    }  # Track minimum costs to each state
    nodes_generated = 1
    nodes_explored = 0

    while frontier:
        _, _, node = heapq.heappop(frontier)
        nodes_explored += 1

        if problem.goal_test(node.state):
            solution_path = solution(node)
            execution_time = format_time(time.perf_counter() - start_time)
            solution_cost = format_time(node.path_cost)
            return solution_path, {
                "nodes_generated": nodes_generated,
                "nodes_explored": nodes_explored,
                "solution_depth": node.depth,
                "solution_cost": solution_cost,
                "execution_time": execution_time,
            }

        explored.add(node.state)

        for next_state, action in node.state.neighbors:
            child_cost = node.path_cost + action.cost()
            if next_state not in explored and (
                next_state not in frontier_state_costs
                or child_cost < frontier_state_costs[next_state]
            ):
                child = Node(
                    next_state,
                    node,
                    action,
                    child_cost,
                    heuristic_cost=heuristic(next_state, problem.goal_state),
                )
                child.f_cost = (
                    child.path_cost + child.heuristic_cost
                )  # Calculate f_cost

                # Push to priority queue based on f_cost and id
                heapq.heappush(frontier, (child.f_cost, child.order, child))
                frontier_state_costs[next_state] = child_cost
                nodes_generated += 1

    execution_time = format_time(time.perf_counter() - start_time)
    return None, {
        "nodes_generated": nodes_generated,
        "nodes_explored": nodes_explored,
        "solution_depth": None,
        "solution_cost": None,
        "execution_time": execution_time,
    }



Implements the A* search algorithm using a priority queue (`heapq`). It utilizes a heuristic function to guide the search towards the goal efficiently.

#### GREEDY BEST-FIRST SEARCH FUNCTION

In [ ]:

def best_first_search(problem, heuristic):
    start_time = time.perf_counter()
    frontier = []
    start_node = Node(
        problem.initial_state,
        heuristic_cost=heuristic(problem.initial_state, problem.goal_state),
    )

    # push tuples con h_cost
    heapq.heappush(
        frontier,
        (
            start_node.heuristic_cost,
            start_node.order,
            start_node,
        ),  # Cambiar id, por tiempo mas viejo (edad)
    )

    explored = set()
    nodes_generated = 1
    nodes_explored = 0

    while frontier:
        _, _, node = heapq.heappop(frontier)
        nodes_explored += 1

        if problem.goal_test(node.state):
            solution_path = solution(node)
            execution_time = format_time(time.perf_counter() - start_time)
            solution_cost = format_time(node.path_cost)
            return solution_path, {
                "nodes_generated": nodes_generated,
                "nodes_explored": nodes_explored,
                "solution_depth": node.depth,
                "solution_cost": solution_cost,
                "execution_time": execution_time,
            }

        explored.add(node.state)

        for next_state, action in node.state.neighbors:
            if next_state not in explored:
                child = Node(
                    next_state,
                    node,
                    action,
                    node.path_cost + action.cost(),
                    heuristic_cost=heuristic(next_state, problem.goal_state),
                )

                # Push with h_cost and id
                heapq.heappush(frontier, (child.heuristic_cost, child.order, child))
                nodes_generated += 1

    execution_time = format_time(time.perf_counter() - start_time)
    return None, {
        "nodes_generated": nodes_generated,
        "nodes_explored": nodes_explored,
        "solution_depth": None,
        "solution_cost": None,
        "execution_time": execution_time,
    }


Implements the Greedy Best-First Search algorithm using a priority queue, guided solely by the heuristic function.

#### HEURISTIC

In [27]:

# Heuristic function for A*
def heuristic(
    state, goal_state
):  # Aqui no debemos usar la distancia sino el tiempo geopy.distance
    #    dx = state.latitude - goal_state.latitude
    #    dy = state.longitude - goal_state.longitude
    #   return (dx**2 + dy**2) ** 0.5
    distance = geodesic(
        (state.latitude, state.longitude), (goal_state.latitude, goal_state.longitude)
    ).meters
    max_speed_kmph = 120
    max_speed_ms = max_speed_kmph / 3.6
    estimated_time = distance / max_speed_ms
    return estimated_time

Calculates the Euclidean distance between the current state and the goal state as the heuristic.

#### ABSTRACTS CLASS OF THE SEARCH ALGORITHMS CALLING THE METHODS

In [28]:

# Abstract Base Class for Search Algorithms
class SearchAlgorithm(ABC):
    @abstractmethod
    def search(self, problem):
        pass


# Breadth-First Search class
class BreadthFirstSearch(SearchAlgorithm):
    def search(self, problem):
        return breadth_first_search(problem)


# Depth-First Search class
class DepthFirstSearch(SearchAlgorithm):
    def search(self, problem):
        return depth_first_search(problem)


# A* Search class
class AStarSearch(SearchAlgorithm):
    def __init__(self, heuristic):
        self.heuristic = heuristic

    def search(self, problem):
        return a_star_search(problem, self.heuristic)


# Best-First Search class
class BestFirstSearch(SearchAlgorithm):
    def __init__(self, heuristic):
        self.heuristic = heuristic

    def search(self, problem):
        return best_first_search(problem, self.heuristic)


We have the Base Class for Search Algorithms `SearchAlgorithm` which defines an abstract base class for all search algorithms, ensuring consistency in their implementation.

The classes `BreadthFirstSearch`, `DepthFirstSearch`, `AStarSearch` and `BestFirstSearch` are concretes classes inheriting from `SearchAlgorithm`, each implementing their respective search strategies.

#### Important Considerations About the Search Class
 - **Use of Node Counter**: In both `a_star_search` and `best_first_search`, a node counter (`Node.node_counter`) is used when pushing nodes onto the priority queue. This ensures that all items in the queue have a unique secondary key, preventing comparison errors when nodes have equal priority values.
 
 - **Explanation of Counter Usage**: The counter is used to avoid redefining comparison methods (`__lt__`, `__gt__`) in the `Node` class. It serves as a tie-breaker in the priority queue when nodes have equal `f_cost` or heuristic cost.
 
 - **Consistent Frontier Management**: The `explored` set and `frontier_state_costs` dictionary are properly updated to manage nodes efficiently.
 
 - **Output Formatting**: The `format_solution_details` function displays the action costs with five decimal places, reflecting the travel time in seconds.
 
 - **Consistent Units**: The `path_cost` and action costs are calculated in seconds, ensuring consistency across the algorithms.
 
 - **Avoiding Comparison Errors**: By using a unique counter when pushing nodes onto the priority queue, the code avoids `TypeError` exceptions that can occur when the heap tries to compare `Node` objects directly.
 
 - **Efficient Frontier Handling**: The use of `frontier_state_costs` in `a_star_search` helps avoid revisiting states via more expensive paths

### CLASS TESTING 

In [33]:
#from ImportJSON import loadJSON
#from Search import (
 #   BreadthFirstSearch,
 #  DepthFirstSearch,
 #   AStarSearch,
 #   BestFirstSearch,
 #   heuristic,
 #   format_solution_details,
#)

# Load the problem from the JSON file
json_file_path = r"C:\Users\andre\DocumentsCopia\ACurso_2425\Primer Cuatri\INTELIGENTES\Lab\Entrega 1\Lab 1. space state search-20241016\examples_with_solutions\problems\huge\calle_agustina_aroca_albacete_5000_0.json"
problem = loadJSON(json_file_path)

# Test BFS
print("Testing BFS:")
bfs_search = BreadthFirstSearch()
bfs_result, bfs_stats = bfs_search.search(problem)
if bfs_result:
    output = format_solution_details(bfs_result, bfs_stats)
    print(output)
else:
    print("No solution found")

# Test DFS
print("\nTesting DFS:")
dfs_search = DepthFirstSearch()
dfs_result, dfs_stats = dfs_search.search(problem)
if dfs_result:
    output = format_solution_details(dfs_result, dfs_stats)
    print(output)
else:
    print("No solution found")

# Test A* Search
print("\nTesting A* Search:")
a_star_search_instance = AStarSearch(heuristic)
print(a_star_search)
a_star_result, a_star_stats = a_star_search_instance.search(problem)
if a_star_result:
    output = format_solution_details(a_star_result, a_star_stats)
    print(output)
else:
    print("No solution found")

# Test Best-First Search
print("\nTesting Best-First Search:")
best_first_search_instance = BestFirstSearch(heuristic)
best_first_result, best_first_stats = best_first_search_instance.search(problem)
if best_first_result:
    output = format_solution_details(best_first_result, best_first_stats)
    print(output)
else:
    print("No solution found")


Testing BFS:
Generated nodes: 785
Expanded nodes: 732
Execution time: 0:00:00.005927
Solution length: 32
Solution cost: 0:06:05.023410
Solution: [1540051673 → 1540051664, 10.02504, 1540051664 → 1526221438, 12.30840, 1526221438 → 1396800476, 9.59340, 1396800476 → 1396800400, 19.95024, 1396800400 → 1526658921, 19.88448, 1526658921 → 1526658962, 18.67404, 1526658962 → 1396800396, 12.38532, 1396800396 → 1396800453, 1.24812, 1396800453 → 1396800406, 1.38564, 1396800406 → 359881802, 2.40000, 359881802 → 1396800416, 1.04652, 1396800416 → 444450205, 0.81408, 444450205 → 1533701521, 18.48114, 1533701521 → 1561396076, 8.40753, 1561396076 → 444448781, 18.88794, 444448781 → 444448782, 3.50748, 444448782 → 444448783, 2.16372, 444448783 → 1578691712, 2.09880, 1578691712 → 444448785, 1.11792, 444448785 → 4221332639, 75.13371, 4221332639 → 9517864886, 0.71832, 9517864886 → 4221332624, 3.61944, 4221332624 → 5062426403, 67.94406, 5062426403 → 4221332646, 1.34820, 4221332646 → 5062426404, 3.74916, 506242

##### Purpose:
* Test eacg search algorithm (BFS, DFS, A*, Best-First) using a problem instance loaded from a JSON file. It runs the algorithms, captures their results and statistics, and prints formatted outputs for verification .

#### Key operations

1. **Loading the problem**:

* Uses `loadJSON` from `ImportJSON.py` to parse the JSON file and initialize the `Problem` instance containing states and actions.

2. **Executing Search Algorithms**
   - **Breadth-First Search (BFS)**:
     - Instantiates `BreadthFirstSearch` and executes the `search` method.
   - **Depth-First Search (DFS)**:
     - Instantiates `DepthFirstSearch` and executes the `search` method.
   - **A\* Search**:
     - Instantiates `AStarSearch` with the provided `heuristic` function and executes the `search` method.
   - **Best-First Search**:
     - Instantiates `BestFirstSearch` with the provided `heuristic` function and executes the `search` method.
     
3. **Output Formatting**:

* Uses `format_solution_details` to neatly format and display the solution paths along with performance statistics such as nodes generated, nodes explored, solution depth, solution cost, and execution time. As well as the action costs with five decimal places, reflecting the travel in seconds.

**Notes**:

- Ensure that the `json_file_path` points to a valid JSON file containing the problem instance.
- The printed outputs provide insights into the efficiency and effectiveness of each search algorithm.